In [42]:
import re
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
import openpyxl
EXCEL_PATH = "/Users/elena/SynologyDrive/Hertie/semester_3/nlp/final_project/geothermie_gesetz_kommentare.xlsx"
SHEET_NAME = "Gesetz + Kommentare"

df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)
df.head()

,Artikel,Typ,Paragraph,Absatz,Gliederungspunkt_Nr,Gesetzestext_Entwurf_1_0307,Gesetzestext_Entwurf_2_1508,Gesetzestext_Entwurf_3_0110,Gesetzestext_Entwurf_4_0312,Org_2,...,Org_24,Org_25,Org_26,Org_27,Org_28,Org_29,Org_30,Org_31,Org_32,Org_33
0,1,Allgemeine Anmerkungen,-1,0,NaN,NaN,NaN,NaN,NaN,Der Beschleunigungseffekt des GeoBG erscheint ...,...,NaN,NaN,NaN,NaN,"Grundsätzlich ist ein Mehr an Geothermie, Sole...",Die Dekarbonisierung der Wärmeversorgung ist e...,s.o. § 1,NaN,NaN,NaN
1,1,Paragraph/Absatz,1,0,NaN,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Paragraph/Absatz,2,0,NaN,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Paragraph/Absatz,3,0,NaN,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Paragraph/Absatz,4,0,NaN,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df.columns

Index(['Artikel', 'Typ', 'Paragraph', 'Absatz', 'Gliederungspunkt_Nr',
       'Gesetzestext_Entwurf_1_0307', 'Gesetzestext_Entwurf_2_1508',
       'Gesetzestext_Entwurf_3_0110', 'Gesetzestext_Entwurf_4_0312', 'Org_2',
       'Org_3', 'Org_5', 'Org_6', 'Org_7', 'Org_8', 'Org_9', 'Org_10',
       'Org_11', 'Org_12', 'Org_13', 'Org_14', 'Org_15', 'Org_16', 'Org_17',
       'Org_18', 'Org_19', 'Org_20', 'Org_21', 'Org_22', 'Org_23', 'Org_24',
       'Org_25', 'Org_26', 'Org_27', 'Org_28', 'Org_29', 'Org_30', 'Org_31',
       'Org_32', 'Org_33'],
      dtype='object')

### Minimal Text Cleaning

In [ ]:
# matches: word-<newline>word
HYPHEN_LINEBREAK_RE = re.compile(r"(\w+)-\s*\n\s*(\w+)", flags=re.UNICODE)

# optional: only if you really want to normalize slashes
SLASH_JOIN_RE = re.compile(r"(\w+)\s*/\s*(\w+)", flags=re.UNICODE)


def clean_legal_text(text: Optional[str]) -> str:
    """
    Cleans legal text while preserving semantic hyphenation.
    Only removes hyphens caused by line breaks (e.g. PDF artifacts).
    """
    if text is None or (isinstance(text, float) and np.isnan(text)):
        return ""

    t = str(text)

    # Normalize line endings
    t = t.replace("\r\n", "\n").replace("\r", "\n")

    # Normalize tabs
    t = t.replace("\t", " ")

    # ✅ Remove hyphenation ONLY when caused by line breaks
    # Example: "Ther-\nmal" -> "Thermal"
    # Keeps: "Erdwärme-Anlage", "CO2-Preis"
    t = HYPHEN_LINEBREAK_RE.sub(r"\1\2", t)

    # Replace remaining newlines with spaces
    t = re.sub(r"\n+", " ", t)

    # Optional: normalize slashed compounds (use with care)
    # Example: "Wärme-/Kältespeicher" -> "Wärme- und Kältespeicher"
    t = SLASH_JOIN_RE.sub(r"\1 und \2", t)

    # Collapse whitespace
    t = re.sub(r"\s+", " ", t).strip()

    return t

In [46]:
VERSION_COLS = [
    'Gesetzestext_Entwurf_1_0307', # Referentenentwurf
    'Gesetzestext_Entwurf_2_1508', # 1. RegE (oder Zwischenstand)
    'Gesetzestext_Entwurf_3_0110', # 2. RegE / BT-Drucksache
    'Gesetzestext_Entwurf_4_0312' # Ausschussbericht / finaler Stand
]

# directly apply cleaning function to every column
df_new = df.copy()
for c in df_new.columns:
    df_new[c] = df_new[c].apply(clean_legal_text)

missing = [c for c in VERSION_COLS if c not in df.columns]
if missing:
    print("Missing columns:", missing)
else:
    for c in VERSION_COLS:
        df[c + "_clean"] = df[c].apply(clean_legal_text)

df[[c + "_clean" for c in VERSION_COLS]].head()

# select only relevant columns starting with "Gesetzestext"
# and drop "Allgemeine Anmkerungen" rows
df_versions = df[df['Typ'] != 'Allgemeine Anmerkungen']
# df_versions = df[df['Typ'] != 'Allgemeine Anmerkungen']#[[c + "_clean" for c in VERSION_COLS]]

# rename columns to simpler names e.g. "'Gesetzestext_Entwurf_1_0307_clean" -> "Gesetzestext_v0_clean"
# keep all columns for now
df_versions = df_versions.rename(columns={
    'Gesetzestext_Entwurf_1_0307_clean': 'Gesetzestext_v0_clean',
    'Gesetzestext_Entwurf_2_1508_clean': 'Gesetzestext_v1_clean',
    'Gesetzestext_Entwurf_3_0110_clean': 'Gesetzestext_v2_clean',
    'Gesetzestext_Entwurf_4_0312_clean': 'Gesetzestext_v3_clean'
})
df_versions.head()

,Artikel,Typ,Paragraph,Absatz,Gliederungspunkt_Nr,Gesetzestext_Entwurf_1_0307,Gesetzestext_Entwurf_2_1508,Gesetzestext_Entwurf_3_0110,Gesetzestext_Entwurf_4_0312,Org_2,...,Org_28,Org_29,Org_30,Org_31,Org_32,Org_33,Gesetzestext_v0_clean,Gesetzestext_v1_clean,Gesetzestext_v2_clean,Gesetzestext_v3_clean
1,1,Paragraph/Absatz,1,0,NaN,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Zweck und Ziel des Gesetzes Zweck dieses Geset...,Zweck und Ziel des Gesetzes Zweck dieses Geset...,Zweck und Ziel des Gesetzes Zweck dieses Geset...,Zweck und Ziel des Gesetzes Zweck dieses Geset...
2,1,Paragraph/Absatz,2,0,NaN,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Anwendungsbereich Dieses Gesetz ist anzuwenden...,Anwendungsbereich Dieses Gesetz ist anzuwenden...,Anwendungsbereich Dieses Gesetz ist anzuwenden...,Anwendungsbereich Dieses Gesetz ist anzuwenden...
3,1,Paragraph/Absatz,3,0,NaN,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Begriffsbestimmungen Im Sinne dieses Gesetzes ...,Begriffsbestimmungen Im Sinne dieses Gesetzes ...,Begriffsbestimmungen Im Sinne dieses Gesetzes ...,Begriffsbestimmungen Im Sinne dieses Gesetzes ...
4,1,Paragraph/Absatz,4,0,NaN,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Überragendes öffentliches Interesse Die Errich...,Überragendes öffentliches Interesse Die Errich...,Überragendes öffentliches Interesse Die Errich...,Überragendes öffentliches Interesse Die Errich...
5,1,Paragraph/Absatz,5,0,NaN,Vorzeitiger Beginn\nFür eine Anlage nach § 2 N...,Vorzeitiger Beginn\nFür eine Anlage nach § 2 N...,Vorzeitiger Beginn\nFür eine Anlage nach § 2 N...,Vorzeitiger Beginn\nFür eine Anlage nach § 2 b...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Vorzeitiger Beginn Für eine Anlage nach § 2 Nu...,Vorzeitiger Beginn Für eine Anlage nach § 2 Nu...,Vorzeitiger Beginn Für eine Anlage nach § 2 Nu...,Vorzeitiger Beginn Für eine Anlage nach § 2 be...


In [48]:
# export as excel for easier reuse
df_new.to_excel("../exports/geobg_cleaned.xlsx", index=False)

### Aggregate text by Artikel (per version)

In [ ]:
def aggregate_by_artikel(
    df: pd.DataFrame,
    artikel_col: str,
    text_col: str,
    sort_cols: list[str] | None = None,
) -> pd.DataFrame:
    """
    Aggregate sub-paragraphs into article-level text.
    """
    d = df.copy()

    if sort_cols:
        sort_cols = [c for c in sort_cols if c in d.columns]
        d = d.sort_values([artikel_col] + sort_cols)

    agg = (
        d.groupby(artikel_col, dropna=False)
         .agg(
             n_subunits=(text_col, lambda x: int(x.notna().sum())),
             text=(text_col, lambda x: "\n".join(
                 [t for t in x if isinstance(t, str) and t.strip()]
             ))
         )
         .reset_index()
    )

    agg["text_len"] = agg["text"].str.len()
    return agg

In [59]:
v0_art = aggregate_by_artikel(
    df,
    artikel_col="Artikel",
    text_col="Gesetzestext_v0_clean",
    sort_cols=["Paragraph", "Absatz"]
)

v1_art = aggregate_by_artikel(
    df,
    artikel_col="Artikel",
    text_col="Gesetzestext_v1_clean",
    sort_cols=["Paragraph", "Absatz"]
)

v3_art = aggregate_by_artikel(
    df,
    artikel_col="Artikel",
    text_col="Gesetzestext_v3_clean",
    sort_cols=["Paragraph", "Absatz"]
)

#### Build article-level comparison tables (v0→v1, v0→v3)

In [66]:
def build_artikel_pairs(a0: pd.DataFrame, aX: pd.DataFrame, suffixX: str):
    out = a0.merge(
        aX,
        on="Artikel",
        how="outer",
        suffixes=("_v0", f"_{suffixX}"),
        indicator=True,
    )

    out["text_v0"] = out["text_v0"].fillna("")
    out[f"text_{suffixX}"] = out[f"text_{suffixX}"].fillna("")

    out["status"] = out["_merge"].map({
        "both": "matched",
        "left_only": "removed",
        "right_only": "added"
    })

    # delete rows where text is empty in both versions
    out = out[~((out["text_v0"] == "") & (out[f"text_{suffixX}"] == ""))]

    out = out.drop(columns="_merge")
    return out


pairs_v01 = build_artikel_pairs(v0_art, v1_art, "v1")
pairs_v03 = build_artikel_pairs(v0_art, v3_art, "v3")


#### Compute similarity (TF–IDF baseline, article level)

In [67]:
def add_tfidf_similarity(pairs: pd.DataFrame, colX: str) -> pd.DataFrame:
    texts = pairs["text_v0"].tolist() + pairs[colX].tolist()

    vec = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=0.95)
    X = vec.fit_transform(texts)

    A = X[: len(pairs)]
    B = X[len(pairs):]

    pairs["sim_tfidf"] = np.diag(cosine_similarity(A, B))
    pairs["delta"] = 1 - pairs["sim_tfidf"]

    return pairs


pairs_v01 = add_tfidf_similarity(pairs_v01, "text_v1")
pairs_v03 = add_tfidf_similarity(pairs_v03, "text_v3")

In [69]:
def summarize(pairs: pd.DataFrame):
    m = pairs[pairs["status"] == "matched"]

    return {
        "N_articles": len(pairs),
        "median_sim": m["sim_tfidf"].median(),
        "p90_delta": m["delta"].quantile(0.90),
        "pct_sim_ge_0_98": (m["sim_tfidf"] >= 0.98).mean() * 100,
    }


summary_v01 = summarize(pairs_v01)
summary_v03 = summarize(pairs_v03)

summary_v01, summary_v03

({'N_articles': 6,
  'median_sim': np.float64(0.9433729220703124),
  'p90_delta': np.float64(0.15018144680649365),
  'pct_sim_ge_0_98': np.float64(16.666666666666664)},
 {'N_articles': 10,
  'median_sim': np.float64(0.8068220396100233),
  'p90_delta': np.float64(1.0),
  'pct_sim_ge_0_98': np.float64(0.0)})

### Similarity at paragraph level (TF-IDF cosine)

In [49]:
@dataclass
class SimilarityConfig:
    ngram_range: Tuple[int, int] = (1, 2)   # unigrams + bigrams
    min_df: int = 1
    max_df: float = 0.95
    use_char_ngrams: bool = True         # set True if you want more robust matching against small edits


def tfidf_cosine_pairwise(a: List[str], b: List[str], cfg: SimilarityConfig) -> np.ndarray:
    if cfg.use_char_ngrams:
        vectorizer = TfidfVectorizer(
            analyzer="char_wb",
            ngram_range=(3, 5),
            min_df=cfg.min_df,
            max_df=cfg.max_df,
        )
    else:
        vectorizer = TfidfVectorizer(
            ngram_range=cfg.ngram_range,
            min_df=cfg.min_df,
            max_df=cfg.max_df,
        )

    # Fit on union so the feature space is shared
    X = vectorizer.fit_transform(a + b)
    Xa = X[: len(a)]
    Xb = X[len(a) :]

    # row-wise cosine: cosine(Xa[i], Xb[i])
    sims = np.array([cosine_similarity(Xa[i], Xb[i])[0, 0] for i in range(Xa.shape[0])])
    return sims


cfg = SimilarityConfig(ngram_range=(1, 2), use_char_ngrams=False)

v0 = df_versions["Gesetzestext_v0_clean"].tolist()
v1 = df_versions["Gesetzestext_v1_clean"].tolist()
v2 = df_versions["Gesetzestext_v2_clean"].tolist()
v3 = df_versions["Gesetzestext_v3_clean"].tolist()

df_versions["sim_v0_v1"] = tfidf_cosine_pairwise(v0, v1, cfg)
df_versions["sim_v1_v2"] = tfidf_cosine_pairwise(v1, v2, cfg)
df_versions["sim_v2_v3"] = tfidf_cosine_pairwise(v2, v3, cfg)
df_versions["sim_v0_v3"] = tfidf_cosine_pairwise(v0, v3, cfg)

df_versions[["sim_v0_v1", "sim_v1_v2", "sim_v2_v3", "sim_v0_v3"]].describe()


,sim_v0_v1,sim_v1_v2,sim_v2_v3,sim_v0_v3
count,51.000000,51.000000,51.000000,51.000000
mean,0.791753,0.909505,0.883173,0.758354
std,0.297655,0.269243,0.269453,0.288963
min,0.000000,0.000000,0.000000,0.000000
25%,0.692913,0.981924,0.933915,0.661603
50%,0.930214,1.000000,1.000000,0.860865
75%,0.986180,1.000000,1.000000,0.937295
max,1.000000,1.000000,1.000000,1.000000


### “Distribution of differences” baseline

In [50]:
THR = 0.95  # tighten/loosen; 0.98 is very strict, 0.90 is looser

df = df_versions.copy()

for a, b in [("v0", "v1"), ("v1", "v2"), ("v2", "v3"), ("v0", "v3")]:
    sim_col = f"sim_{a}_{b}"
    df[f"delta_{a}_{b}"] = 1.0 - df[sim_col]
    df[f"changed_{a}_{b}"] = df[sim_col] < THR

summary = pd.DataFrame({
    "transition": ["v0→v1", "v1→v2", "v2→v3", "v0→v3"],
    "n_paragraphs": [len(df)] * 4,
    "share_changed(<thr)": [
        df["changed_v0_v1"].mean(),
        df["changed_v1_v2"].mean(),
        df["changed_v2_v3"].mean(),
        df["changed_v0_v3"].mean(),
    ],
    "median_delta(1-sim)": [
        df["delta_v0_v1"].median(),
        df["delta_v1_v2"].median(),
        df["delta_v2_v3"].median(),
        df["delta_v0_v3"].median(),
    ],
    "p90_delta": [
        df["delta_v0_v1"].quantile(0.90),
        df["delta_v1_v2"].quantile(0.90),
        df["delta_v2_v3"].quantile(0.90),
        df["delta_v0_v3"].quantile(0.90),
    ],
})

summary

,transition,n_paragraphs,share_changed(<thr),median_delta(1-sim),p90_delta
0,v0→v1,51,0.627451,6.978602e-02,0.477166
1,v1→v2,51,0.215686,3.330669e-16,0.084219
2,v2→v3,51,0.333333,1.332268e-15,0.259849
3,v0→v3,51,0.764706,1.391353e-01,0.541013


### 4.2 Top changed paragraphs (for qualitative inspection)

In [51]:
KEY_COLS = [c for c in ["Artikel", "Paragraph", "Absatz", "Gliederungspunkt_Nr"] if c in df.columns]

top = (
    df.sort_values("delta_v0_v3", ascending=False)
      .loc[:, KEY_COLS + ["delta_v0_v3", "sim_v0_v3", "Gesetzestext_v0_clean", "Gesetzestext_v3_clean"]]
      .head(15)
)

top

,Artikel,Paragraph,Absatz,Gliederungspunkt_Nr,delta_v0_v3,sim_v0_v3,Gesetzestext_v0_clean,Gesetzestext_v3_clean
53,10,0,0,0.0,1.000000,0.000000,,Änderung des Wärmeplanungsgesetzes Das Wärmepl...
52,9,0,0,0.0,1.000000,0.000000,,Änderung des Baugesetzbuches Das Baugesetzbuch...
51,8,0,0,0.0,1.000000,0.000000,,Änderung der Verordnung zur Anrechnung von str...
34,4,1,0,1.0,1.000000,0.000000,,1. In der Inhaltsübersicht wird die Angabe zu ...
50,7,0,0,0.0,1.000000,0.000000,,Änderung des Bundes-Immissionsschutzgesetzes D...
39,4,6,0,6.0,0.541013,0.458987,5. § 56 wird wie folgt geändert: a) Nach Absat...,6. § 56 wird wie folgt geändert: a) Nach § 56 ...
12,1,8,0,NaN,0.456556,0.543444,"Planfeststellung, Plangenehmigung","Planfeststellung, Plangenehmigung, Enteignungs..."
37,4,4,0,4.0,0.429076,0.570924,3. In § 51 Absatz 3 werden nach den Angaben „v...,4. In § 51 Absatz 3 Satz 1 wird nach der Angab...
10,1,7,1,NaN,0.407643,0.592357,(1) Eigentümer und sonstige Nutzungsberechtigt...,(1) Eigentümer und sonstige Nutzungsberechtigt...
36,4,3,0,3.0,0.406390,0.593610,2. § 15 wird wie folgt geändert: a) Die Angabe...,3. § 15 wird durch den folgenden § 15 ersetzt:...


In [36]:
extra_v3 = df[
    (df["Gesetzestext_v3_clean"].str.len() > 0)
    & (df["Gesetzestext_v0_clean"].str.len() == 0)
]

extra_v3[KEY_COLS + ["Gesetzestext_v3_clean"]].head(20)

,Artikel,Paragraph,Absatz,Gliederungspunkt_Nr,Gesetzestext_v3_clean
34,4,1,0,1.0,1. In der Inhaltsübersicht wird die Angabe zu ...
50,7,0,0,0.0,Änderung des Bundes-Immissionsschutzgesetzes D...
51,8,0,0,0.0,Änderung der Verordnung zur Anrechnung von str...
52,9,0,0,0.0,Änderung des Baugesetzbuches Das Baugesetzbuch...
53,10,0,0,0.0,Änderung des Wärmeplanungsgesetzes Das Wärmepl...


In [37]:
mask_aligned = df["Gesetzestext_v0_clean"].str.len() > 0
df_aligned = df[mask_aligned].copy()

df_aligned["sim_v0_v3"].describe()

count    46.000000
mean      0.840784
std       0.148094
min       0.458987
25%       0.765023
50%       0.884708
75%       0.947244
max       1.000000
Name: sim_v0_v3, dtype: float64

In [52]:
sim = df_aligned["sim_v0_v3"]

summary_v0_v3 = {
    "mean": sim.mean(),
    "median": sim.median(),
    "p90": sim.quantile(0.90),
    "share_above_0.98": (sim > 0.98).mean(),
}

summary_v0_v3


{'mean': np.float64(0.8407835054879034),
 'median': np.float64(0.8847081305470474),
 'p90': np.float64(1.0),
 'share_above_0.98': np.float64(0.17391304347826086)}

In [55]:
delta_v0_v3 = 1 - df_aligned["sim_v0_v3"]
delta_v0_v3.describe()

count    4.600000e+01
mean     1.592165e-01
std      1.480935e-01
min     -4.440892e-16
25%      5.275580e-02
50%      1.152919e-01
75%      2.349769e-01
max      5.410129e-01
Name: sim_v0_v3, dtype: float64

In [56]:
THR = 0.98

changed = df_aligned[df_aligned["sim_v0_v3"] < THR]
unchanged = df_aligned[df_aligned["sim_v0_v3"] >= THR]

{
    "pct_changed": len(changed) / len(df_aligned),
    "median_changed": changed["sim_v0_v3"].median(),
    "median_unchanged": unchanged["sim_v0_v3"].median(),
}


{'pct_changed': 0.8260869565217391,
 'median_changed': np.float64(0.8553418113667385),
 'median_unchanged': np.float64(1.0)}